In [1]:
import os
import json
import random
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset

In [2]:
fnames = [
'Sc_psd_7hz_results.json',
'VR_psd_7hz_results.json',
'Sc_psd_10hz_results.json',
'VR_psd_10hz_results.json',
'Sc_psd_12hz_results.json',
'VR_psd_12hz_results.json',
'Sc_psd_all_results.json',
'VR_psd_all_results.json']

In [3]:
caseList = ["user_mix", "user_specific", "user_novel"]

In [4]:
fullRes = []

for case in caseList:
    results = []
    for fname in fnames:
        platform, feature, sti = fname.replace("_results.json", "").split("_")
        result= json.load(open(os.path.join(f"../ssvl/act_cls/svm/exp/{case}", fname), 'r'))
        instance = {
            "platform": platform,
            "sti": sti
        }
        instance.update(result['score'])
        results.append(instance)
        
    fullRes.append(results)

In [5]:
pd.DataFrame(fullRes[1])

,platform,sti,avg_train,avg_test
0,Sc,7hz,0.920023,0.759722
1,VR,7hz,0.886227,0.678704
2,Sc,10hz,0.918981,0.743981
3,VR,10hz,0.892245,0.697222
4,Sc,12hz,0.909722,0.771759
5,VR,12hz,0.907407,0.712037
6,Sc,all,0.893017,0.779012
7,VR,all,0.866049,0.717130


In [27]:
import statistics as stat

caseList = ["user_specific", "user_novel"]
results = []

for case in caseList:
    for fname in fnames:
        platform, feature, sti = fname.replace("_results.json", "").split("_")
        result= json.load(open(os.path.join(f"../ssvl/act_cls/svm/exp/{case}", fname), 'r'))      

        trainRes = []
        testRes = []
        pList = ['P01', 'P02', 'P04', 'P05', 'P06', 'P07', 'P08', 'P09', 'P10','P11','P12','P13', 'P14', 'P15','P16','P17','P18','P20']
        for p in pList:
            res = [{k:v} for k,v in result.items() if p in k]

            # case 1인 경우
            if case == 'user_specific':
                trainScr = res[0][p + '_fold_wise']['train_score']
                testScr = res[0][p + '_fold_wise']['test_score']

            # case 2인 경우
            elif case == 'user_novel':
                trainScr = res[0][p + '_test_fold']['train_score']
                testScr = res[0][p + '_test_fold']['test_score']

            #print(trainScr)

            trainRes.append(trainScr)
            testRes.append(testScr)
            
        stdRes = [case, platform, sti, stat.stdev(trainRes), stat.stdev(testRes)]
        results.append(stdRes)


pd.DataFrame(results)


,0,1,2,3,4
0,user_specific,Sc,7hz,0.098901,0.053101
1,user_specific,VR,7hz,0.140588,0.054172
2,user_specific,Sc,10hz,0.104741,0.048931
3,user_specific,VR,10hz,0.141969,0.066712
4,user_specific,Sc,12hz,0.139037,0.064205
5,user_specific,VR,12hz,0.144683,0.054796
6,user_specific,Sc,all,0.110944,0.058412
7,user_specific,VR,all,0.139108,0.066868
8,user_novel,Sc,7hz,0.005108,0.087970
9,user_novel,VR,7hz,0.004506,0.089455
